# Data source

Data for download: https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

Resources used as reference:
1) Training the image classifier to recognize different species of flowers:
https://www.kaggle.com/dtosidis/flower-classifier-tensorflow
  
2) Loading and preprocessing an image dataset
https://www.tensorflow.org/tutorials/load_data/images

3) Data augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

4) Image classification
https://www.tensorflow.org/tutorials/images/classification


In [1]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf

import PIL
import PIL.Image

import pathlib

os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import (
    VGG19, 
    preprocess_input, 
    decode_predictions
)

In [2]:
!pwd

/c/ShankersDocs/EDUCATION/RICE_Bootcamp_DataAnalytics/FinalProject_Img_Recognition_Flowers/Final_RICEproject_ImageRecognition_flowers


### Total images in the dataset

In [3]:
data_dir = 'flower_photos'
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
# print(image_count)

## Image Classification
https://www.tensorflow.org/tutorials/images/classification?hl=zh-tw

In [4]:

batch_size = 32
img_height = 180
img_width = 180

## Generating datasets
https://keras.io/examples/vision/image_classification_from_scratch/

### Generating a training dataset

In [5]:
# When the subset below is defined as "training" the 0.2 validation split takes 80% of the data as the training set

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


### Generating a validation dataset

In [6]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the validation set

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 367 files for validation.


### Generating a test dataset

In [7]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the test set

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 367 files for validation.


### Class names

In [8]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


### Rescaling the data
https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling


In [9]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

### Normalizing the data (trainign and validation datasets)

In [10]:
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds =  val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.0 1.0


### Autotune is done to cache data and make processing and resource mgmt more effieicient

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
normalized_train_ds = normalized_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalized_val_ds = normalized_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [13]:
# num_classes = 5
num_classes = len(class_names)
num_classes

5

## Model 1 (Sequential Model)
https://www.tensorflow.org/guide/keras/sequential_model

In [23]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [24]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [27]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=16
)

Epoch 1/16
92/92 [==============================] - 43s 470ms/step - loss: 870.0703 - accuracy: 0.3563 - val_loss: 1174.6035 - val_accuracy: 0.3270
Epoch 2/16
92/92 [==============================] - 6s 65ms/step - loss: 655.5504 - accuracy: 0.4077 - val_loss: 440.5868 - val_accuracy: 0.4251
Epoch 3/16
92/92 [==============================] - 6s 65ms/step - loss: 472.6843 - accuracy: 0.4387 - val_loss: 447.1398 - val_accuracy: 0.4060
Epoch 4/16
92/92 [==============================] - 6s 65ms/step - loss: 482.6045 - accuracy: 0.4458 - val_loss: 673.5029 - val_accuracy: 0.3869
Epoch 5/16
92/92 [==============================] - 6s 66ms/step - loss: 290.2948 - accuracy: 0.4898 - val_loss: 466.4365 - val_accuracy: 0.3460
Epoch 6/16
92/92 [==============================] - 6s 65ms/step - loss: 239.0393 - accuracy: 0.4898 - val_loss: 289.1722 - val_accuracy: 0.4414
Epoch 7/16
92/92 [==============================] - 6s 67ms/step - loss: 256.2381 - accuracy: 0.4721 - val_loss: 808.0683 - val

## Model 1 (Sequential Model) Epoch sensitivity

In [28]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [29]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [30]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=6
)

Epoch 1/6
92/92 [==============================] - 6s 68ms/step - loss: 3109.6846 - accuracy: 0.3198 - val_loss: 2002.3905 - val_accuracy: 0.3324
Epoch 2/6
92/92 [==============================] - 6s 64ms/step - loss: 1059.0967 - accuracy: 0.3740 - val_loss: 1328.4467 - val_accuracy: 0.2425
Epoch 3/6
92/92 [==============================] - 6s 65ms/step - loss: 676.3892 - accuracy: 0.4217 - val_loss: 1104.6982 - val_accuracy: 0.3324
Epoch 4/6
92/92 [==============================] - 6s 67ms/step - loss: 480.6086 - accuracy: 0.4561 - val_loss: 832.1366 - val_accuracy: 0.3569
Epoch 5/6
92/92 [==============================] - 6s 66ms/step - loss: 406.3172 - accuracy: 0.4935 - val_loss: 830.6475 - val_accuracy: 0.3706
Epoch 6/6
92/92 [==============================] - 6s 66ms/step - loss: 464.5512 - accuracy: 0.4816 - val_loss: 1717.7488 - val_accuracy: 0.2834


## Model 1a (Sequential Model) With Normalized data

In [40]:
model_1a = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [41]:
model_1a.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [42]:
model_1a.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=6
)

Epoch 1/6
92/92 [==============================] - 6s 67ms/step - loss: 9.3329 - accuracy: 0.2548 - val_loss: 1.5924 - val_accuracy: 0.2834
Epoch 2/6
92/92 [==============================] - 6s 66ms/step - loss: 1.5261 - accuracy: 0.3392 - val_loss: 1.5553 - val_accuracy: 0.3406
Epoch 3/6
92/92 [==============================] - 6s 65ms/step - loss: 1.5153 - accuracy: 0.3236 - val_loss: 1.5620 - val_accuracy: 0.3651
Epoch 4/6
92/92 [==============================] - 6s 65ms/step - loss: 1.4879 - accuracy: 0.3304 - val_loss: 1.5920 - val_accuracy: 0.4033
Epoch 5/6
92/92 [==============================] - 6s 65ms/step - loss: 1.4419 - accuracy: 0.3675 - val_loss: 1.5167 - val_accuracy: 0.4142
Epoch 6/6
92/92 [==============================] - 6s 68ms/step - loss: 1.4356 - accuracy: 0.3658 - val_loss: 1.5409 - val_accuracy: 0.4142


## Model 2 

In [14]:
model_2 = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [15]:
model_2.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [16]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=6
)

Epoch 1/6
92/92 [==============================] - 106s 1s/step - loss: 1.4361 - accuracy: 0.3713 - val_loss: 1.1264 - val_accuracy: 0.5095
Epoch 2/6
92/92 [==============================] - 99s 1s/step - loss: 1.0575 - accuracy: 0.5790 - val_loss: 1.1003 - val_accuracy: 0.5668
Epoch 3/6
92/92 [==============================] - 97s 1s/step - loss: 0.9199 - accuracy: 0.6492 - val_loss: 0.9367 - val_accuracy: 0.6349
Epoch 4/6
92/92 [==============================] - 79s 864ms/step - loss: 0.7503 - accuracy: 0.7204 - val_loss: 0.9182 - val_accuracy: 0.6540
Epoch 5/6
92/92 [==============================] - 98s 1s/step - loss: 0.5740 - accuracy: 0.7933 - val_loss: 0.9419 - val_accuracy: 0.6567
Epoch 6/6
92/92 [==============================] - 101s 1s/step - loss: 0.4128 - accuracy: 0.8481 - val_loss: 1.0187 - val_accuracy: 0.6376


In [18]:
model_2.save('model_2')

NotFoundError: Failed to create a NewWriteableFile: model_2\variables\variables_temp_fce808fd195e414ba3479b096bf087db/part-00000-of-00001.data-00000-of-00001.tempstate9719282433297583692 : The system cannot find the path specified.
; No such process [Op:SaveV2]